# Tutorial 1 for R

## Solve Dantzig's Transport Problem using the *ix modeling platform* (ixmp)

<img style="float: right; height: 80px;" src="_static/R_logo.png">

### Aim and scope of the tutorial

This tutorial takes you through the steps to import the data for a very simple optimization model
and solve it using the ``ixmp``-GAMS interface.

We use Dantzig's transport problem, which is also used as the standard GAMS tutorial.
This problem finds a least cost shipping schedule that meets requirements at markets and supplies at factories.
If you are not familiar with GAMS, please take a minute to look at the [transport.gms](transport.gms) code.

For reference of the transport problem, see:
> Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions.  
> Princeton University Press, Princeton, New Jersey, 1963.

> This formulation is described in detail in:  
> Rosenthal, R E, Chapter 2: A GAMS Tutorial.  
> In GAMS: A User's Guide. The Scientific Press, Redwood City, California, 1988.

> see http://www.gams.com/mccarl/trnsport.gms

The steps in the tutorial are the following:

0. Launch an ixmp.Platform instance and initialize a new ixmp.Scenario.
0. Define the sets and parameters in the scenario, and commit the data to the platform
0. Check out the scenario and initialize variables and equations (necessary for ``ixmp`` to import the solution)
0. Solve the model (export to GAMS input gdx, execute, read solution from output gdx)
0. Display the solution (variables and equation)

### Launching the platform and initializing a new scenario

We launch a platform instance and initialize a new scenario. This will be used to store all data required to solve Dantzig's transport problem as well as the solution after solving it in GAMS.

In [ ]:
# Load reticulate, used to access the Python API from R
library(reticulate)

# Import ixmp and message_ix, just as in Python
ixmp <- import("ixmp")

In [ ]:
# launch the ix modeling platform using a local HSQL database instance
mp <- ixmp$Platform()

In [ ]:
# details for creating a new scenario in the ix modeling platform         
model <- "transport problem" 
scenario  <- "standard"
annot <- "Dantzig's transportation problem for illustration and testing" 

# initialize a new ixmp.Scenario
# the parameter version='new' indicates that this is a new scenario instance
scen <- ixmp$Scenario(mp, model, scenario, "new", annotation=annot)

### Defining the sets in the scenario

Below, we first show the data as they would be written in the GAMS tutorial ([transport.gms](transport.gms) in this folder).  
Then, we show how this can be implemented in the R ``ixmp`` notation, and display the elements of set ``i`` as an R list.

In [ ]:
# define the sets of locations of canning plants  
scen$init_set("i")
i.set = c("seattle","san-diego")
scen$add_set("i", i.set )

### markets set
scen$init_set("j")
j.set = c("new-york","chicago","topeka")
scen$add_set("j", j.set )

In [ ]:
# display the set 'i'
scen$set('i')

### Defining parameters in the scenario

Next, we define the production capacity and demand parameters, and display the demand parameter as a DataFrame.
Then, we add the two-dimensional distance parameter and the transport cost scalar.

In [ ]:
# capacity of plant i in cases                                                                                                                                                           
scen$init_par("a", c("i"))
a.df = data.frame( i = i.set, value = c(350 , 600) , unit = 'cases')
scen$add_par("a", a.df)
#scen$add_par("a", "san-diego", 600, "cases")

# demand at market j in cases
scen$init_par("b", c("j"))
b.df = data.frame( j = j.set, value = c(325 , 300, 275) , unit = 'cases')
scen$add_par("b", b.df)

In [ ]:
# display the parameter 'b'
scen$par('b')

In [ ]:
# distance in thousands of miles
scen$init_par("d", c("i","j"))
d.df = data.frame(expand.grid(i = i.set,j = j.set), value = c(2.5,2.5,1.7,1.8,1.8,1.4), unit = 'km')
scen$add_par("d", d.df)

Scalar f  freight in dollars per case per thousand miles  /90/ ; 

In [ ]:
# cost per case per 1000 miles
# initialize scalar with a value and a unit (and optionally a comment)
scen$init_scalar("f", 90.0, "USD/km")

### Committing the scenario to the ixmp database instance

In [ ]:
# commit new scenario to the database
# no changes can then be made to the scenario data until a check-out is performed
comment = "importing Dantzig's transport problem for illustration of the R interface"
scen$commit(comment)

# set this new scenario as the default version for the model/scenario name
scen$set_as_default()

### Defining variables and equations in the scenario

The levels and marginals of these variables and equations will be imported to the scenario when reading the gdx solution file.

In [ ]:
# perform a check_out to make further changes
scen$check_out()

# initialize the decision variables and equations
scen$init_var("z", NULL, NULL)
scen$init_var("x", idx_sets=c("i", "j"))
scen$init_equ("demand", idx_sets=c("j"))

# commit changes to the scenario (save changes in ixmp database instance)
change_comment = "inialize the model variables and equations"
scen$commit(change_comment)

### Solve the scenario

The ``solve()`` function exports the scenario to a GAMS gdx file, executes GAMS, and then imports the solution from an output GAMS gdx file to the database.

For the model equations and the GAMS workflow (reading the data from gdx, solving the model, writing the results to gdx), see ``transport_ixmp.gms``.

In [ ]:
scen$solve(model='dantzig')

### Display and analyze the results

In [ ]:
# display the objective value of the solution
scen$var("z")

In [ ]:
# display the quantities transported from canning plants to demand locations
scen$var("x")

In [ ]:
# display the quantities and marginals (=shadow prices) of the demand balance constraints
scen$equ("demand")

### Close the database connection of the ix modeling platform

Closing the database connection is recommended when working with the local file-based database, i.e., ``dbtype='HSQLDB'``. This command closes the database files and removes temporary data. This is necessary so that other notebooks or ``ixmp`` instances can access the database file, or so that the database files can be copied to a different folder or drive.

In [ ]:
# close the connection of the platform instance to the local database files
mp$close_db()